# NHL Data Analysis
  "If you have time can you put like an intro here? Doesnt have to be long"
### Research Questions
    - Which NHL Teams have been statistcally the most dominate compared to achieved results
    - For each NHL Team, who has had the most "Successful Tenure"
    - "Vanessa fill in"
    - "Vanessa Fill in"
---
Please view both of our EDA's beforehand for a little more context and understanding of what will be doing!

### Loading Preliminary functions

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas_profiling
from pandas_profiling import ProfileReport
import sys
sys.path.append('..')
from scripts import ProjectFunctions as pf

### Importing Data Lists
The next line we are just importing the data from our local repositorys, this can be changed to a url or any other method for replicaton
- The [NHL DATA is from this Kaggle page.](https://www.kaggle.com/martinellis/nhl-game-data) for convienance

All data is provided and can be sourced from the NHL’s API, unofficial documentation can be found on Drew Hynes’ gitlab here: https://gitlab.com/dword4/nhlapi/-/blob/master/stats-api.md

In [ ]:
k = "C:/Users/keati/Documents/Cosc301/Nhl_Data/game_teams_stats.csv"
l = "C:/Users/keati/Documents/Cosc301/Nhl_Data/team_info.csv"

df1 = pf.CoachData(k)
df2 = pf.TeamData(k)
df3 = pf.TeamList(l)

Coach_df = pd.merge(df3,df1, how='inner')
total_df = pd.concat([df3,df2],axis=1).reindex(df2.index)

## Successful Tenure

To start off our analysis of the most dominate team I first wanted to look at each individual team and figure out who on which coach has had the best run on each team

In [ ]:
Total = Coach_df.sort_values('won').drop_duplicates('teamName', keep = 'last').reset_index(drop=True)

- From this dataframe I was able to plot and see which had the most amount of wins with each individual team

In [ ]:
Most_wins = Total.sort_values('won').drop_duplicates('teamName', keep = 'last').reset_index(drop=True)
sns.barplot(x='won',y='head_coach',data=Most_wins,linewidth=1)
plt.xlabel("Wins")
plt.ylabel("Head Coach")
plt.title("Most Wins for each Team")

Moving on from this base Information I wanted to see what made each coach with there respective made them so successful.

To achieve this I created another two columns figuring out the each teams total goal percentage and power play goal percentage

In [ ]:
Total['Goal Percentage in %'] = ((Total['goals']/Total['shots'])*100)
Total['PPG Goal Percentage in %'] = ((Total['powerPlayGoals']/Total['powerPlayOpportunities'])*100)
Total.head()

In [ ]:
sns.barplot(x='Goal Percentage in %',y='head_coach',data=Total,linewidth=0.2)

In [ ]:
sns.barplot(x='PPG Goal Percentage in %',y='head_coach',data=Total,linewidth=0.2)

## Most dominate

To talk about most dominate, We will be exploring the stats from each team regardless off there overall achieved results in the last 8 years

From this we are able to see which teams have been able to win the most games with the bruins almost doubling the Jets. We also must acknlowedge the data may be a bit flawed from this perspective as the Golden Knights have not been in the league as long as every other team.

In [ ]:
total_df.head()

In [ ]:
pd_df = total_df.sort_values(['won']).reset_index(drop=True)
sns.barplot(x='won',y='teamName',data=pd_df,linewidth=1)

To work around this I am going to create a weighted total including shots, hits, takeaways and blocked shots as these stats are more relative to the overal play of a team rather than goals.
The scores will be computed with:
 - 40% Shots - Help a team win  with offense
 - 30% takeaway - Generates a turn over and gives the team the play
 - 10% Hits - defensive statistic, defending your zone
 - 20% block shots - defensive statistic
 
 

In [ ]:
total_df['Weighted Score'] = (0.4*df2['shots'] + 0.3*df2['takeaways'] + 0.1*df2['hits'] + 0.2*df2['blocked'])
Sample = pd.DataFrame(total_df.groupby("teamName")['Weighted Score'].mean().sort_values(ascending=False).reset_index())
Sample

This produces an interesting result. You can see that Bruins are the top when it comes to the weighted total, however, the wild are near the bottom however only finishing middle of the pack in wins

## Vanessa Research Question

## Vanessa Research Question 2